In [ ]:
%matplotlib inline

from tiptop import *

In [ ]:
fullInputPath = "localTest/"
parametersFile = 'params1'
windPsdFile = 'MASTSEL/data/windpsd_mavis.fits'
baseOutpuPath = 'localTest/'
outputFile = "test"
overallSimulation(fullInputPath, parametersFile, windPsdFile, baseOutpuPath, outputFile, doConvolve=True, pitchScaling = 0.9)

In [ ]:
# MAVIS SIMULATIONS
#baseInputPath = '/data7/guido/results/MAVIS/res4psf/20201116_111006.0_oaPSFs_w'
#baseOutpuPath = '/data4/olivierbm/phaseB/fourierVe2e/MAVIS/fourierHO_LO/'
#windPsdFile = 'data/windpsd_mavis.fits'
baseInputPath = '/data7/guido/results/MAVIS/res4psf/20201116_111006.0_oaPSFs_w'
baseOutpuPath = '/data4/frossi/'
windPsdFile = 'MASTSEL/data/windpsd_mavis.fits'

wvlPaths = ['450','550','640','890']
k=-1
for wvlPath in wvlPaths:
    k+=1
    fullInputPath = baseInputPath + wvlPath + 'nm/'
    parametersFile = 'params'
    outputFile = 'simulation' + wvlPath + 'nm'
    overallSimulation(fullInputPath, parametersFile, windPsdFile, baseOutpuPath, outputFile,doConvolve=True,pitchScaling = 0.9)
print('Done')